In [ ]:
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
import torch
import torchvision
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms
import numpy as np
from sklearn.preprocessing import scale
from torch.optim.lr_scheduler import StepLR

In [ ]:
class LeNet(nn.Module):
    """
    X = к-во входных каналов
    
    Input - Xx32x32
    Conv1 - 6 фильтров, 5x5 ядро
    ReLU
    Pool2 - 6 фильтров, 2х2 ядро отступ 2
    Conv3 - 16 фильтров 5x5 ядро
    ReLU
    Pool4 - 16  фильтров, 2х2 ядро отступ 2
    Conv5 - 120 фильтров,  5x5 ядро
    F6 - 84
    ReLU
    F7 - 10 
    """
    def __init__(self, in_channel = 1):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channel, 6,5 )
        self.relu = nn.ReLU()
        self.pool2 = nn.MaxPool2d(2,stride = 2)
        self.conv3 = nn.Conv2d(6, 16, 5)
        self.pool4 = nn.MaxPool2d(2, stride = 2)
        self.conv5 = nn.Conv2d(16, 120, 5)
        self.f6 = nn.Linear(120, 84)
        self.f7 = nn.Linear(84, 10)
    
    def forward(self,x):
        result = self.relu(self.conv1(x))
        result = self.relu(self.conv3(self.pool2(result)))
        result = self.conv5(self.pool4(result))
        result = result.view(-1, 120)
        result = self.f7(self.relu(self.f6(result)))
        return result
LeNet(3)

In [ ]:
data_train = torchvision.datasets.CIFAR10('./pytorch_data/cifar10', 
                                          download=True, 
                                          transform=transforms.Compose([transforms.Resize((32,32)), 
                                                                        transforms.ToTensor()]))
    
data_test = torchvision.datasets.CIFAR10('./pytorch_data/cifar10', 
                                         train=False, 
                                         download=True, 
                                         transform=transforms.Compose([transforms.Resize((32,32)), 
                                                                       transforms.ToTensor()]))

data_train_loader = DataLoader(data_train, batch_size=128, shuffle=True, num_workers=8)
data_test_loader = DataLoader(data_test, batch_size=128, num_workers=8)

In [ ]:
device = ("cuda:0" if torch.cuda.is_available() else "cpu")

def accuracy(output,labels):
    predictions = torch.argmax(output,dim=1)
    correct = (predictions == labels).sum().cpu().numpy()
    return correct / len(labels)

loss_func = torch.nn.CrossEntropyLoss()

EPOCH_SIZE = 20
device = torch.device('cuda:0' if torch.cuda else 'cpu')

model = LeNet(3)

optimizer = torch.optim.SGD(model.parameters(), lr=1e-2, nesterov = True, momentum = 0.9)

In [ ]:
model.train()
for epoch in range(EPOCH_SIZE):
    for itr,data in enumerate(data_train_loader):
        imgs = data[0]
        labels = data[1]

    
        y_pred = model.forward(imgs)

        optimizer.zero_grad()
    
        loss = loss_func(y_pred,labels)
        if itr%100 == 0:
            print('Iteration {}, train accuracy {:.2f}, loss {:.4f}'.format(itr+epoch*len(data_train_loader),accuracy(y_pred,labels),loss))
      
        loss.backward()
    
        optimizer.step()

In [ ]:
model.eval()

with torch.no_grad():
  accuracy_list = []
  for itr,data in enumerate(data_test_loader):
    imgs = data[0]
    labels = data[1]
    
    y_pred = model.forward(imgs)
    accuracy_list.append(accuracy(y_pred,labels))

print('Test accuracy - {:.2f}'.format(np.sum(accuracy_list)/len(accuracy_list)))